In [1]:
import sys  
sys.path.insert(0, './Druida_POC/src/')


from druida import Stack
from druida import setup
from druida.DataManager import datamanager

import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn

from torchsummary import summary
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms

from torchvision.transforms import ToTensor, Lambda

from matplotlib import pyplot as plt


random.seed(1)

In [2]:
#dnn.clear()
#dnn.pop()
#dnn.forward()
#dnn.layers

trainer = Stack.Trainer(learning_rate=0.001,batch_size=1, epochs=5)

dnn=Stack.DNN([{"name":"hidden1","layer":nn.Linear(28*28, 512), "type":"hidden", "index":0},
               {"name":"activation1","layer":nn.ReLU(),"type":"activation", "index":1},
               {"name":"hidden2","layer":nn.Linear(512,512),"type":"hidden", "index":2},
               {"name":"activation2","layer":nn.ReLU(),"type":"activation", "index":3}, 
               {"name":"hidden3","layer":nn.Linear(512,10),"type":"hidden", "index":4}])

##checking device 
dnn.device
dnn.to(dnn.device)

DNN(
  (architecture): Sequential(
    (hidden1): Linear(in_features=784, out_features=512, bias=True)
    (activation1): ReLU()
    (hidden2): Linear(in_features=512, out_features=512, bias=True)
    (activation2): ReLU()
    (hidden3): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [3]:
#training_data = datasets.FashionMNIST(
#    root="data",
#    train=True,
#    download=True,
#    transform=ToTensor(),
#    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
#)

#test_data = datasets.FashionMNIST(
#    root="data",
#    train=False,
#    download=True,
#    transform=ToTensor(),
#    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))   
#)

#train_dataloader = DataLoader(training_data, batch_size=trainer.batch_size)
#test_dataloader = DataLoader(test_data, batch_size=trainer.batch_size)


#for x, y in train_dataloader:
#    print(x.size())
#    print(y.size())
#    break


In [6]:
image_size = 32
DATA_DIR = '/data/Cactus/aerial-cactus-identification/train/'
cwd = os.getcwd() 




#Crea el dataset. Usa "dataset.imgs" para mostrar el nombre del archivo
training_data = datasets.ImageFolder(root=cwd+DATA_DIR,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize([0.5],[0.5]) 
                           ]))
#Crea el dataloader
train_dataloader = torch.utils.data.DataLoader(training_data, batch_size=trainer.batch_size,
                                         shuffle=False, num_workers=2)



for x, y in train_dataloader:
    print(x)
    print(y)
    break

tensor([[[[ 0.0667,  0.0510,  0.1608,  ...,  0.1922,  0.1451,  0.2235],
          [-0.0275,  0.2627,  0.2471,  ...,  0.2784,  0.1529,  0.1373],
          [ 0.2627,  0.2549,  0.2471,  ...,  0.1765,  0.2627,  0.0118],
          ...,
          [ 0.0745,  0.5451, -0.1529,  ...,  0.2471,  0.1373,  0.2235],
          [ 0.1765,  0.3255,  0.1765,  ...,  0.3176,  0.1922,  0.1922],
          [ 0.4353, -0.0980,  0.2941,  ...,  0.4039,  0.1137,  0.0902]],

         [[ 0.0824,  0.0667,  0.1608,  ...,  0.0118, -0.0196,  0.0588],
          [-0.0118,  0.2784,  0.2471,  ...,  0.0980, -0.0118, -0.0275],
          [ 0.2784,  0.2706,  0.2471,  ...,  0.0118,  0.0980, -0.1529],
          ...,
          [-0.1294,  0.3412, -0.3333,  ...,  0.0353, -0.0745,  0.0118],
          [-0.0275,  0.1216, -0.0275,  ...,  0.0902, -0.0353, -0.0353],
          [ 0.2314, -0.3020,  0.0902,  ...,  0.1765, -0.1137, -0.1373]],

         [[-0.0196, -0.0353,  0.0980,  ...,  0.0588,  0.0196,  0.0980],
          [-0.1137,  0.1765,  

In [1]:
#labels_map = {
#    0: "T-Shirt",
#    1: "Trouser",
#    2: "Pullover",
#    3: "Dress",
#    4: "Coat",
#    5: "Sandal",
#    6: "Shirt",
#    7: "Sneaker",
#    8: "Bag",
#    9: "Ankle Boot",
#}
# Display image and label.
# train_features, train_labels = next(iter(train_dataloader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")
# img = train_features[0].squeeze()
# label = train_labels[0]
# plt.imshow(img, cmap="gray")
# plt.show()
# print(f"Label: {label}")

In [14]:

#optimizer
optimizer = torch.optim.Adam(dnn.parameters(), lr=trainer.learning_rate )
#loss function
loss_fn = nn.CrossEntropyLoss()

In [15]:
#Arrays to log and plot loss values



def train_loop(dataloader, model, loss_fn, optimizer, batch, acc,loss_val):
    size = len(dataloader)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()

    X,y=dataloader
    
    size = len(X)


    # Compute prediction and loss

    #prediction
    pred = model(X)
    loss = loss_fn(pred, y)
    optimizer.zero_grad()

    # Backpropagation
    loss.backward()
    optimizer.step()
        
    running_loss = loss.item()
    current =  (batch + 1) * len(X)
    #print(f"Training Loss: {loss:>7f}  [{current:>5d}/{size:>5d}] \n")
    acc = (pred.argmax(1) == y).type(torch.float).mean().item()

    print(f"Training: \n Accuracy: {100*acc}%, Loss: {loss:>7f}  [{current:>5d}/{size:>5d}] \n")
    
    return acc, running_loss

    



def test_loop(dataloader, model, loss_fn,batches, batch,acc_test,test_loss):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    num_batches = batches

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        X, y = (dataloader)
        size = len(X)

        pred = model(X)
        test_loss = loss_fn(pred, y).item()
        acc_test = (pred.argmax(1) == y).type(torch.float).mean().item()
           
    
    #test_loss /= num_batches
    print(f"Test Error: \n Accuracy: {100*(acc_test):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    return acc_test, test_loss

In [16]:
trainer.training(train_loop,test_loop, train_dataloader,test_dataloader, dnn, loss_fn, optimizer)

Epoch 1
-------------------------------


RuntimeError: mat1 and mat2 shapes cannot be multiplied (28x28 and 2352x512)